<a href="https://colab.research.google.com/github/atejap05/pos_data_science_furg/blob/main/disciplinas/Machine_Learning_I/semana08/Furg_ECD_08_Machine_Learning_I_Redes_neurais_(parte_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Redes Neurais (parte 2)
### Prof. Marcelo Malheiros

Código adaptado de Aurélien Geron (licença Apache-2.0)

---

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_:

- NumPy: suporte a vetores, matrizes e operações de Álgebra Linear
- Matplotlib: biblioteca de visualização de dados
- Pandas: pacote estatístico e de manipulação de DataFrames
- Scikit-Learn: biblioteca com algoritmos de Machine Learning

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

Este _notebook_, em particular, utiliza a biblioteca Keras para definir e treinar redes neurais. Aqui utilizamos a versão **integrada** de Keras, que já vem como parte da biblioteca mais geral TensorFlow.

Em geral é mais fácil usar a versão integrada de Keras, pois esta está pronta para usar a Tensorflow, sem risco de incompatibilidade.

Ambas já fazem parte do ambiente Colaboratory.

**Atenção:** para quem utiliza o ambiente Jupyter, é preciso primeiro instalar o pacote `tensorflow`. Na linha de comando isso pode ser feito assim:

    conda install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras

print('tensorflow:      versão', tf.__version__)
print('keras integrada: versão', keras.__version__)

Este _notebook_ também utiliza a biblioteca `pydot` e a ferramenta Graphviz para visualizar as redes neurais. 

Ambos já fazem parte do ambiente Colaboratory.

**Atenção:** para quem utiliza o ambiente Jupyter, é preciso primeiro instalar os pacotes `pydot` e `graphviz`. Na linha de comando isso pode ser feito assim:

    conda install pydot graphviz

In [ ]:
import pydot

# Construindo um regressor MLP

Aqui vamos usar o conjunto clássico de dados sobre habitação na Califórnia para implementar um regressor usando redes neurais.

Mais especificadamente, vamos usar a versão deste _dataset_ provida pela biblioteca Sciki-Learn através da função `fetch_california_housing()`. São 20640 instâncias, cada uma com 8 _features_ e mais um rótulo. Este conjunto contém apenas valores numéricos (ou seja, sem dados categóricos) e também não há valores faltantes.

Os passos iniciais de carregamento, separação em conjuntos (treino, validação e teste) e um rápido preprocessamento (apenas escalonando os valores para a mesma faixa numérica) são mostrados a seguir.

In [ ]:
# leitura do dataset

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

# separação inicial em conjuntos de treino completo (75%) e de teste (25%)

from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target,
                                                              random_state=42, train_size=0.75)

# separação posterior do conjuntos de treino completo em dados de treino (75%) e de validação (25%)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full,
                                                      random_state=42, train_size=0.75)

# preprocessamento

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # a escala é definida e então a transformação é feita
X_valid = scaler.transform(X_valid)     # apenas a transformação é feita (com a escala previamente definida)
X_test  = scaler.transform(X_test)      # apenas a transformação é feita (com a escala previamente definida)

In [ ]:
print('treino:   ', X_train.shape,  ' rótulos de treino:   ', y_train.shape)
print('validação:', X_valid.shape, '  rótulos de validação:', y_valid.shape)
print('teste:    ', X_test.shape,  '  rótulos de teste:    ', y_test.shape)

# Criando a rede neural

Aqui vamos criar uma rede neural de regressão, usando um modelo (ou arquitetura) do tipo sequencial. O modelo sequencial corresponde ao tipo mais simples de rede neural, onde uma sequência de camadas de neurônios é empilhada uma em cima da outra.

É importante também observar que o conjunto de dados `housing` tem **muito ruído**. Então é preferível usarmos apenas uma camada oculta com poucos neurônios, justamente para evitar overfitting. Em outras palavras, optamos por uma arquitetura de rede neural bem simples.

- A criação começa com a chamada a `Sequential`, que define o tipo do modelo:

        model = keras.models.Sequential()

- Então uma camada do tipo `Dense` é adicionada, que é do tipo totalmente conectada com a camada anterior. Esta conta com 30 neurônios e função de ativação ReLU. Como esta é a primeira camada, é preciso também definir o formato da entrada com `input_shape` (neste caso, com 8 entradas):

        model.add(keras.layers.Dense(30, activation='relu', input_shape=[8]))
        
- Daí podemos adicionar uma camada de saída, com tipo também `Dense`, mas sem função de ativação (portanto com o parâmetro `None`). A ideia é que o único neurônio da camada não tenha nenhum tipo de modificação, e justamente produza a previsão numérica deste regressor:
        
        model.add(keras.layers.Dense(1, activation=None))

Ainda que se possa criar uma rede neural com as diversas chamadas a `model.add(...)`, é mais conveniente criar o modelo passando uma lista de camadas, como mostrado a seguir.

In [ ]:
# comando para 'zerar' a biblioteca Keras
keras.backend.clear_session()

# definição de sementes aleatórias
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# especificação do modelo
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    keras.layers.Dense(1, activation=None)
])

In [ ]:
# resumo legível da arquitetura deste modelo
model.summary()

Note que a primeira camada densa tem pesos de conexão de 8 × 30, além de mais 30 termos de _bias_, chegando a um total de 270 parâmetros.

## Arquitetura da rede neural

Podemos gerar uma figura da arquitetura deste modelo usando a função `keras.utils.plot_model`.

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

## Acesso às camadas

A biblioteca Keras permite acessar cada camada criada, usando índices de acesso tal como em uma lista de Python.

Permite também ver atributos de cada camada e também inspecionar os pesos de todas as conexões desta.

In [ ]:
# acesso a cada uma das camadas
model.layers

In [ ]:
# primeira camada e respectivo nome
hidden1 = model.layers[0]
hidden1.name

Observe que uma camada `Dense` começa com pesos aleatórios para suas conexões. Os vieses são sempre inicializados com zero.

In [ ]:
# obtém pesos e vieses da primeira camada
weights, biases = hidden1.get_weights()
print('weights:', weights.shape)
print('biases: ', biases.shape)

In [ ]:
weights

In [ ]:
biases

## Compilando a rede neural

Depois que um modelo é criado, é preciso chamar o método `compile()`, especificando a **função de perda**. Como é uma tarefa de regressão, deseja-se reduzir o erro quadrado médio, então a função é `mean_squared_error`.

Como é típico para redes neurais, o treinamento será feito usando **backpropagation**. Nesse caso, continuamos usando `sgd` como **otimizador**, ou seja, o algoritmo de descida do gradiente estocástico.

Como o erro médio quadrado é justamente a nossa medida do quão bom será o treino, validação e teste, não é necessário definir nenhuma outra medida de desempenho.

In [ ]:
# especificação da função de perda e do algoritmo de otimização
model.compile(loss='mean_squared_error', optimizer='sgd')

# Treinando e avaliando a rede neural

Para treinar o modelo basta chamar o método `fit()`. 

Três parâmetros são obrigatórios: as _features_ de treinamento, os rótulos de treinamento e o número de épocas.

Cada **época** (_epoch_) corresponde a uma etapa de atualização da rede neural.

Opcionalemente é passado também um conjunto de validação. A biblioteca Keras medirá a função de perda (o erro, neste caso) ao final de cada época, o que é muito útil para ver como o modelo realmente funciona: se o desempenho no conjunto de treinamento é muito melhor do que no conjunto de validação, provavelmente está ocorrendo _overfitting_.

In [ ]:
# treinamento (esta chamada pode demorar um pouco)
%time history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Agora, após o treinamento, vamos examinar os pesos e vieses ajustados.

In [ ]:
# obtém pesos e vieses da primeira camada
weights, biases = hidden1.get_weights()
weights

In [ ]:
biases

## Avaliação final do modelo e geração de previsões

In [ ]:
# avaliação com conjunto de teste
model.evaluate(X_test, y_test)

In [ ]:
# previsões computadas para três instâncias de teste
X_new = X_test[:3]
y_pred = model.predict(X_new)
print('previsões: ', y_pred.ravel().round(3))
print('rótulos:   ', y_test[:3].ravel().round(3))

## Visualização da evolução das métricas ao longo do treinamento

In [ ]:
# os dados do treinamento estão disponíveis no histórico retornado
print('parâmetros:', history.params)
print('métricas:  ', list(history.history.keys()))
print('épocas:    ', history.epoch)

In [ ]:
# exibição das funções de perda de treino e de validação, para cada época (eixo horizontal)
pd.DataFrame(history.history).plot(figsize=(10, 4))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

# Gravação e recuperação de um modelo treinado

Uma vez que o modelo tenha sido treinado e testado, ele está pronto para o uso. Isso permite inclusive que ele possa ser salvo em um arquivo e utilizado posteriormente, sem necessidade de repetir todo o treinamento.

A gravação do modelo é feita pela função `.save()` do modelo, que armazena todas informações do mesmo em um único arquivo.

Em qualquer momento posterior, tipicamente em programas diferentes, este arquivo pode ser carregado para a memória com o comando `keras.models.load_model()`, permitindo que o modelo seja reconstruído exatamente como antes.

**Dica:** a biblioteca TensorFlow pode gerar uma advertência caso muitas chamadas sejam feitas para a função `.predict()`. A ideia da biblioteca é alertar que muitas "previsões pequenas" possam ser ineficientes, mas isto aqui não é problema. Para que não sejam geradas essas advertências, basta converter as instâncias a serem previstas usando `tf.constant()` antes de chamar a predição, como é exemplificado abaixo.

In [ ]:
# algumas previsões para o modelo já treinado
y_pred = model.predict(tf.constant(X_test[:10]))
print('previsões: ', y_pred.ravel().round(3))

In [ ]:
# '.h5' é a extensão do formato Hierarchical Data Format versão 5 (HDF5)
model.save('housing_model.h5')

In [ ]:
# posteriormente o modelo pode ser reconstruído (aqui em outro objeto 'model2')
model2 = keras.models.load_model('housing_model.h5')

In [ ]:
# mesmas previsões para o novo modelo recuperado do arquivo
y_pred2 = model2.predict(tf.constant(X_test[:10]))
print('previsões: ', y_pred2.ravel().round(3))

# Construindo diferentes arquiteturas com Keras

Existem três maneiras de criar modelos de redes neurais com a biblioteca Keras:

- A forma mais direta é com a **Sequential API**, onde a arquitetura é justamente uma sequência de camadas. A principal limitação é ter uma única camada de entrada e uma única camada de saída.

- Keras também provê a **Functional API**, que é uma API de programação que permite especificar modelos mais complexos e com arquiteturas arbitrárias.

- Ainda é possível programar modelos "do zero" com a **Subclassing API**, usando o conceito de subclasses e implementando manualmente os detalhes de uma arquitetura. Assim é possível criar modelos dinâmicos, que podem mudar ao longo do tempo.

A seguir vamos apenas ilustrar a especificação de algumas arquiteturas usando a API Funcional, sem entrar em maiores detalhes.

## Exemplo 1 - Wide and Deep Neural Network

In [ ]:
keras.backend.clear_session()

input_A = keras.layers.Input(shape=[8])
hidden1 = keras.layers.Dense(30, activation='relu')(input_A)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat  = keras.layers.concatenate([input_A, hidden2])
output  = keras.layers.Dense(1)(concat)
example = keras.models.Model(inputs=[input_A], outputs=[output])

In [ ]:
example.summary()

In [ ]:
keras.utils.plot_model(example, show_shapes=True)

## Exemplo 2 - Múltiplas entradas

In [ ]:
keras.backend.clear_session()

input_A = keras.layers.Input(shape=[6], name='wide_input')
input_B = keras.layers.Input(shape=[5], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_A)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat  = keras.layers.concatenate([input_B, hidden2])
output  = keras.layers.Dense(1, name='output')(concat)
example = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [ ]:
keras.utils.plot_model(example, show_shapes=True)

## Exemplo 3 - Múltiplas entradas e múltiplas saídas

In [ ]:
keras.backend.clear_session()

input_A = keras.layers.Input(shape=[6], name='wide_input')
input_B = keras.layers.Input(shape=[5], name='deep_input')
hidden1 = keras.layers.Dense(30, activation='relu')(input_A)
hidden2 = keras.layers.Dense(30, activation='relu')(hidden1)
concat  = keras.layers.concatenate([input_B, hidden2])
output1 = keras.layers.Dense(1, name='main_output')(concat)
output2 = keras.layers.Dense(1, name='aux_output')(hidden2)
example = keras.models.Model(inputs=[input_A, input_B], outputs=[output1, output2])

In [ ]:
keras.utils.plot_model(example, show_shapes=True)

# Visualização usando TensorBoard

TensorBoard é uma excelente ferramenta de visualização interativa, muito útil para analisar as curvas de aprendizado do treinamento.

Esta ferramenta também permite comparar curvas de aprendizado entre várias execuções, analisar estatísticas de treinamento, visualizar imagens geradas pelo modelo e inspecionar detalhes internos do modelo, por exemplo.

In [ ]:
# comando para 'zerar' a biblioteca Keras
keras.backend.clear_session()

# definição de sementes aleatórias
np.random.seed(42)
tf.random.set_seed(42)

# especificação do modelo
model1 = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    keras.layers.Dense(1, activation=None)
])

# especificação da função de perda e do algoritmo de otimização
model1.compile(loss='mean_squared_error', optimizer='sgd')

TensorBoard faz parte da biblioteca TensorFlow, portanto não é necessária nenhuma instalação adicional.

É necessario apenas modificar o programa para que ele armazene os dados a serem visualizados em arquivos binários de _log_. A melhor estratégia é criar um diretório principal, e dentro deste um subdiretório para cada execução do treinamento. Assim é possível comparar visualmente diferentes rodadas de treino.

No código abaixo, na chamada de treinamento adicionamos o parâmetro `callbacks=[tensorboard]`, justamente para que o TensorBoard tenha acesso aos dados. Adicionalmente, usamos o parâmetro `verbose=0` para não exibir as barras de progresso.

In [ ]:
# define o diretório principal para os logs

import os
log_dir = os.path.join(os.curdir, 'logs')

# define o subdiretório de treinamento com base na data e hora atuais

import time
run_dir1 = os.path.join(log_dir, time.strftime('run_%Y_%m_%d-%H_%M_%S'))

# vincula o tensorboard ao subdiretório de treinamento e depois faz o treino

tensorboard1 = keras.callbacks.TensorBoard(run_dir1)
%time history1 = model1.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), \
                            callbacks=[tensorboard1], verbose=0)

Tanto o ambiente Colaboratory como a ferramenta Jupyter possuem suporte para integrar o TensorBoard dentro de um _notebook_.

É justamente isso que fazem os comandos a seguir, lendo os _logs_ a partir do diretório principal `logs`. Nesta primeira execução, só há um subdiretório.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./logs --port=6006

Agora vamos criar um segundo modelo, mudando apenas o hiperparâmetro de taxa de aprendizagem (_learning rate_) do algoritmo SGD.

Quando treinarmos este segundo modelo, um novo subdiretório será criado dentro de `logs`, e então podemos atualizar o painel do TensorBoard acima para visualizar e comparar as duas curvas de treinamento.

In [ ]:
model2 = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    keras.layers.Dense(1, activation=None)
])

model2.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(lr=0.05))

run_dir2 = os.path.join(log_dir, time.strftime('run_%Y_%m_%d-%H_%M_%S'))

tensorboard2 = keras.callbacks.TensorBoard(run_dir2)
%time history2 = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), \
                            callbacks=[tensorboard2], verbose=0)

# Ajuste de hiperparâmetros de redes neurais

Para um modelo construído com a **Sequential API**, os principais hiperparâmetros são:

- O número de camadas ocultas.

- O número de neurônios em cada camada oculta.

- A taxa de aprendizado (_learning rate_), que é o hiperparâmetro mais importante. O padrão para o SGD é 0.01.

- O otimizador utilizado, uma vez que há outras opções além do SGD.

- O tamanho do lote de treinamento (_batch size_), usualmente 32.

- A função de ativação (_activation function_). Tipicamente a função ReLU é usada para neurônios ocultos. Funções específicas para cada tarefa (regressão ou classificação) são usadas na camada de saída, assim como uma função de perda adequada.

- O número de épocas de treinamento.

O código abaixo permite experimentar explicitamente com alguns destes parâmetros, ainda usando o _dataset_ `housing`. Cabe destacar o parâmetro `callbacks = [keras.callbacks.EarlyStopping(patience=10)]`, que diz que o treinamento pode parar mais cedo caso não haja redução da função de perda por 10 épocas seguidas.

In [ ]:
neurons        = 30     # 10 30 50
activation     = 'relu' # 'relu' 'sigmoid' 'tanh'
learning_rate  = 0.01   # 0.005 0.01 0.05
batch_size     = 32     # 16 32 64
epochs         = 20     # 10 20 50 100
callbacks      = []     # [keras.callbacks.EarlyStopping(patience=10)]

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(neurons, activation=activation, input_shape=[8]),
#   keras.layers.Dense(neurons, activation=activation),
    keras.layers.Dense(1, activation=None)
])

model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(lr=learning_rate))

%time history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), \
                          batch_size=batch_size, epochs=epochs, callbacks=callbacks, verbose=0)

pd.DataFrame(history.history).plot(figsize=(10, 4))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

print('erro de treino:   ', history.history['loss'][-1])
print('erro de validação:', history.history['val_loss'][-1])
print('erro de teste:    ', model.evaluate(X_test, y_test, verbose=0))